In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    import zipfile
    with zipfile.ZipFile('/content/drive/MyDrive/DL Project/YoloV5DataSet.zip', 'r') as zip_ref:
        zip_ref.extractall('./')
except:
    print("Using Local Machine")

In [1]:
import sys
!git clone https://github.com/ultralytics/yolov5.git
# sys.path.append('/path/to/yolov5')
!pip install -r yolov5/requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 15627, done.
remote: Counting objects: 100% (234/234), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 15627 (delta 116), reused 138 (delta 66), pack-reused 15393
Receiving objects: 100% (15627/15627), 14.65 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (10644/10644), done.
zsh:1: command not found: pip


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from yolov5.models.yolo import Model
from yolov5.utils.dataloaders import LoadImagesAndLabels
from yolov5.utils.general import check_requirements, set_logging
from yolov5.utils.loss import ComputeLoss
from yolov5.utils.plots import plot_labels, plot_results
from pathlib import Path
import yaml

import os
import random
from pathlib import Path
import pandas as pd
import cv2


In [ ]:
batchSize = 32
inputShape = (640, 640)
epochs = 100
numAnchors = 3
yolo5Version = 'm'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using {} device".format(device))


In [ ]:
# Load custom dataset configuration
with open('signboard.yaml') as f:
    data = yaml.safe_load(f)
train_path, val_path = data['train'], data['val']
nc, names = data['nc'], data['names']

In [ ]:
# Create output directory
save_dir = Path('runs/train/signboard_exp')
(save_dir / 'weights').mkdir(parents=True, exist_ok=True)
set_logging(str(save_dir / 'train.log'))


In [ ]:
def custom_collate_fn(batch):
    # Get the maximum length of tensors in the batch
    max_len = max([t.shape[0] for _, t, _, _ in batch])

    # Pad the tensors to the same size and stack them together
    imgs = torch.stack([img for img, _, _, _ in batch])
    targets = [torch.cat([t, torch.zeros(max_len - t.shape[0], t.shape[1])], dim=0) for _, t, _, _ in batch]
    targets = torch.stack(targets)
    paths = [path for _, _, path, _ in batch]
    extra = [e for _, _, _, e in batch]

    return imgs, targets, paths, extra

In [ ]:
# Load dataset
train_dataset = LoadImagesAndLabels(train_path, 640, 16, rect=True, pad=0.5)
val_dataset = LoadImagesAndLabels(val_path, 640, 16, rect=True, pad=0.5)

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=custom_collate_fn,shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=16, collate_fn=custom_collate_fn,shuffle=False, num_workers=4, pin_memory=True)


In [ ]:
print("Downloading Weights of yolo5 Verion ", yolo5Version)
weightsURL = "https://github.com/ultralytics/yolov5/releases/download/v5.0/yolov5{}.pt".format(yolo5Version)
!wget {weightsURL}

In [ ]:
def CreateYolov5Model(numClasses: int, version: str):
    congfigFile = "yolov5/models/yolov5{}.yaml".format(version)
    model = Model(congfigFile, ch=3, nc=numClasses)
    ckpt = torch.load(f'yolov5{version}.pt', map_location=device)
    ckpt_model_dict = ckpt['model'].state_dict()
    compatible_weights = {k: v for k, v in ckpt_model_dict.items() if k in model.state_dict() and model.state_dict()[k].shape == v.shape}
    model.load_state_dict(compatible_weights, strict=False)
    model.hyp = ckpt['model'].hyp
    return model


In [ ]:

def TargetstoTensors(targets, batchSize, numAnchors, gridSizes):
    targetObj = []
    targetBox = []
    for grid_size in gridSizes:
        targetObj.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 1)))
        targetBox.append(torch.zeros((batchSize, numAnchors, grid_size, grid_size, 4)))

    for batch_index, target in enumerate(targets):
        x1, y1, x2, y2 = target.long()
        x_center, y_center, width, height = (x1 + x2) / 2, (y1 + y2) / 2, x2 - x1, y2 - y1

        for i, grid_size in enumerate(gridSizes):
            x_cell, y_cell = int(x_center * grid_size), int(y_center * grid_size)
            anchor = 0
            try:
                targetObj[i][batch_index, anchor, y_cell, x_cell, 0] = 1
                targetBox[i][batch_index, anchor, y_cell, x_cell] = torch.tensor([x_center, y_center, width, height])
            except Exception as e:
                pass
    return targetObj, targetBox

In [ ]:

class SignboardLoss(nn.Module):
    def __init__(self, num_anchors=3):
        super(SignboardLoss, self).__init__()
        self.num_anchors = num_anchors

    def forward(self, preds, targets):
        objectLoss = torch.tensor(0.0, device=preds[0].device)
        boxLoss = torch.tensor(0.0, device=preds[0].device)
        batchSize = preds[0].size(0)
        gridSizes = [pred.size(2) for pred in preds]
        targetObjList, targetBoxList = TargetstoTensors(targets, batchSize, self.num_anchors, gridSizes)

        for i, pred in enumerate(preds):
            targetObj = targetObjList[i].to(pred.device)
            targetBox = targetBoxList[i].to(pred.device)

            objectLoss += nn.BCEWithLogitsLoss()(pred[..., 4:5], targetObj)
            boxLoss += nn.MSELoss()(pred[..., :4], targetBox)

        total_loss = objectLoss + boxLoss
        return total_loss

In [ ]:
# Load model
numClasses = 47
model = CreateYolov5Model(numClasses,yolo5Version)
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.937, weight_decay=0.0005, nesterov=True)
criterion = SignboardLoss()
criterion = criterion.to(device)


In [ ]:

epochs = 100
best_fitness = float('inf')

for epoch in range(epochs):
    model.train()

    for i, (imgs, targets, paths, _) in enumerate(train_loader):
        imgs = imgs.to(device).float() / 255.0
        targets = targets.to(device)

        # Forward pass
        pred = model(imgs)
        print("Pred shapes:")
        for p in pred:
            print(p.shape)
        print("Targets shape:", targets.shape)
        # Compute loss
        loss, loss_items = criterion(pred, targets)
        loss_items = torch.cat(loss_items)

        # Backward pass
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

    # Validation
    model.eval()
    with torch.no_grad():
        for i, (imgs, targets, paths, _) in enumerate(val_loader):
            imgs = imgs.to(device).float() / 255.0
            targets = targets.to(device)

            # Forward pass
            pred = model(imgs)
            print("Pred shape:", pred.shape)
            print("Targets shape:", targets.shape)
            # Compute loss
            val_loss, val_loss_items = criterion(pred, targets)
            val_loss_items = torch.cat(val_loss_items)

    # Save best model
    if val_loss < best_fitness:
        best_fitness = val_loss
        torch.save(model.state_dict(), save_dir / 'weights' / 'best.pt')

# Plot training results
plot_results(save_dir=save_dir)
